In [2]:
import pandas as pd 
import duckdb 


In [3]:
green = duckdb.connect('green_taxi.db')

In [7]:
conn = duckdb.connect('green_taxi.db')
green_df = conn.execute("SELECT * FROM read_csv('green_tripdata_2019-10.csv') ").fetch_df()
green_df = pd.DataFrame(green_df)
green_df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2019-10-01 00:26:02,2019-10-01 00:39:58,N,1.0,112,196,1.0,5.88,18.0,0.50,0.5,0.00,0.0,None,0.3,19.30,2.0,1.0,0.0
1,1.0,2019-10-01 00:18:11,2019-10-01 00:22:38,N,1.0,43,263,1.0,0.80,5.0,3.25,0.5,0.00,0.0,None,0.3,9.05,2.0,1.0,0.0
2,1.0,2019-10-01 00:09:31,2019-10-01 00:24:47,N,1.0,255,228,2.0,7.50,21.5,0.50,0.5,0.00,0.0,None,0.3,22.80,2.0,1.0,0.0
3,1.0,2019-10-01 00:37:40,2019-10-01 00:41:49,N,1.0,181,181,1.0,0.90,5.5,0.50,0.5,0.00,0.0,None,0.3,6.80,2.0,1.0,0.0
4,2.0,2019-10-01 00:08:13,2019-10-01 00:17:56,N,1.0,97,188,1.0,2.52,10.0,0.50,0.5,2.26,0.0,None,0.3,13.56,1.0,1.0,0.0


In [10]:
green_df.tail()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
476381,NaN,2019-10-31 23:30:00,2019-11-01 00:00:00,None,NaN,65,102,NaN,7.04,29.57,2.75,0.5,0.0,0.00,None,0.0,32.82,NaN,NaN,NaN
476382,NaN,2019-10-31 23:03:00,2019-10-31 23:24:00,None,NaN,129,136,NaN,0.00,39.83,2.75,0.5,0.0,6.12,None,0.0,49.20,NaN,NaN,NaN
476383,NaN,2019-10-31 23:02:00,2019-10-31 23:23:00,None,NaN,61,222,NaN,3.90,23.11,2.75,0.5,0.0,0.00,None,0.0,26.36,NaN,NaN,NaN
476384,NaN,2019-10-31 23:42:00,2019-10-31 23:56:00,None,NaN,76,39,NaN,3.08,15.23,2.75,0.5,0.0,0.00,None,0.0,18.48,NaN,NaN,NaN
476385,NaN,2019-10-31 23:23:00,2019-10-31 23:56:00,None,NaN,56,215,NaN,6.84,36.12,2.75,0.5,0.0,0.00,None,0.3,39.67,NaN,NaN,NaN


In [9]:
green_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476386 entries, 0 to 476385
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   VendorID               387007 non-null  float64       
 1   lpep_pickup_datetime   476386 non-null  datetime64[us]
 2   lpep_dropoff_datetime  476386 non-null  datetime64[us]
 3   store_and_fwd_flag     387007 non-null  object        
 4   RatecodeID             387007 non-null  float64       
 5   PULocationID           476386 non-null  int64         
 6   DOLocationID           476386 non-null  int64         
 7   passenger_count        387007 non-null  float64       
 8   trip_distance          476386 non-null  float64       
 9   fare_amount            476386 non-null  float64       
 10  extra                  476386 non-null  float64       
 11  mta_tax                476386 non-null  float64       
 12  tip_amount             476386 non-null  floa

**3. Trip segmentation count**

In [11]:
october = green_df[(green_df['lpep_pickup_datetime'] >= '2019-10-01') & 
                   (green_df['lpep_dropoff_datetime'] < '2019-11-01')]

In [12]:
counts = {
    "Up to 1 mile": october[october['trip_distance'] <= 1.0],
    "Between 1 and 3 miles": october[(october['trip_distance'] > 1.0) & (october['trip_distance'] <= 3.0)],
    "Between 3 and 7 miles": october[(october['trip_distance'] > 3.0) & (october['trip_distance'] <= 7.0)],
    "Between 7 and 10 miles": october[(october['trip_distance'] > 7.0) & (october['trip_distance'] <= 10.0)],
    "Over 10 miles": october[october['trip_distance'] > 10.0]
}

for range_name, filtered_data in counts.items():
    print(f"{range_name}: {filtered_data.shape[0]} trips")


Up to 1 mile: 104802 trips
Between 1 and 3 miles: 198924 trips
Between 3 and 7 miles: 109603 trips
Between 7 and 10 miles: 27678 trips
Over 10 miles: 35189 trips


**4. Longest trip for each day**

In [18]:
distance = october['trip_distance'].max()
day = october[october['trip_distance'] == distance]

day 



,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
141174,2.0,2019-10-11 20:34:21,2019-10-11 22:40:41,N,4.0,126,265,5.0,95.78,473.0,0.5,0.5,0.0,10.5,None,0.3,484.8,2.0,1.0,0.0


**5. Three biggest pickup Zones**

In [35]:
october['lpep_pickup_datetime'] = pd.to_datetime(october['lpep_pickup_datetime'])

target_date = pd.to_datetime('2019-10-18').date()

dates = october[october['lpep_pickup_datetime'].dt.date == target_date]
dates 

C:\Users\PC\AppData\Local\Temp\ipykernel_12428\391343748.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  october['lpep_pickup_datetime'] = pd.to_datetime(october['lpep_pickup_datetime'])


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
216483,2.0,2019-10-18 23:35:11,2019-10-19 00:09:42,N,1.0,247,48,1.0,6.34,21.00,0.50,0.5,0.00,0.00,None,0.3,22.30,2.0,1.0,0.0
216611,2.0,2019-10-18 00:00:40,2019-10-18 00:04:41,N,1.0,145,112,5.0,0.84,5.50,0.50,0.5,1.36,0.00,None,0.3,8.16,1.0,1.0,0.0
216664,2.0,2019-10-18 00:00:10,2019-10-18 00:13:16,N,1.0,95,215,1.0,3.38,13.00,0.50,0.5,0.00,0.00,None,0.3,14.30,2.0,1.0,0.0
216864,2.0,2019-10-18 00:01:17,2019-10-18 00:09:42,N,1.0,82,82,1.0,1.67,8.00,0.50,0.5,0.00,0.00,None,0.3,9.30,2.0,1.0,0.0
216930,2.0,2019-10-18 22:58:52,2019-10-19 00:07:30,N,1.0,74,116,1.0,7.69,31.50,0.50,0.5,6.56,0.00,None,0.3,39.36,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434209,NaN,2019-10-18 23:20:00,2019-10-19 00:00:00,None,NaN,129,211,NaN,8.29,26.28,5.50,0.5,0.00,6.12,None,0.0,38.40,NaN,NaN,NaN
434210,NaN,2019-10-18 23:39:00,2019-10-19 00:12:00,None,NaN,180,193,NaN,10.29,32.53,0.00,0.5,0.00,0.00,None,0.0,33.03,NaN,NaN,NaN
434211,NaN,2019-10-18 23:59:00,2019-10-19 00:17:00,None,NaN,47,127,NaN,3.71,22.25,2.75,0.5,0.00,0.00,None,0.0,25.50,NaN,NaN,NaN
434212,NaN,2019-10-18 23:03:00,2019-10-18 23:15:00,None,NaN,169,18,NaN,2.32,17.39,2.75,0.5,0.00,0.00,None,0.0,20.64,NaN,NaN,NaN


In [30]:
locations = dates.groupby('PULocationID')['total_amount'].agg('sum').sort_values(ascending=False).reset_index()
locations 


,PULocationID,total_amount
0,74,18686.68
1,75,16797.26
2,166,13029.79
3,41,12440.66
4,82,12431.96
...,...,...
231,128,24.03
232,96,21.30
233,184,12.80
234,57,10.30


From the data dictionary provide in the **TLC NYC** website, the top areas include:
* East Harlem North
* East Harlem South
* Morningside Heights 


**6. Largest Tip**

In [36]:
harlem = green_df[green_df['PULocationID'] == 74]
tips = harlem.groupby('DOLocationID')['tip_amount'].agg('sum').sort_values(ascending=False).reset_index()
tips 

,DOLocationID,tip_amount
0,236,4935.36
1,75,4076.44
2,166,3369.36
3,263,3322.15
4,238,2942.32
...,...,...
208,192,0.00
209,190,0.00
210,203,0.00
211,207,0.00
